In [2]:
import os
import re
from bs4 import BeautifulSoup

ext_html = re.compile("^.*\.html$")

res = [ data for data in os.listdir(".") if ext_html.match(data) ]

#admitidos_file = open('RMT.html')
admitidos_file = open("RHT-2015.html")
admitidos_content = admitidos_file.read()
admitidos_file.close()

admitidos_soup = BeautifulSoup(admitidos_content, "html.parser")

# Obtener los nombres de las carreras o preparatorias
# print(admitidos_soup.prettify())
titles = admitidos_soup.find_all("h2")
if len(titles) > 1: 
  try:
    print(titles[1].small.contents[0].strip())
  except AttributeError:
    print(titles[0].u.contents[0])
else:
  print(titles[0].u.contents[0])

INGENIERIA AGROHIDRAULICA (TECAMACHALCO)


In [92]:
results_dict = {}
prepas_re = re.compile("PREPARATORIA|PREP")

for results in res:
    with open(results) as r:
        res_soup = BeautifulSoup(r.read(), "html.parser")
        
        titles = res_soup.find_all("h2")
        name = ""
        try:
            name = titles[1].small.contents[0].strip()
        except (AttributeError, IndexError):
            name = titles[0].u.contents[0]
        
        results_dict[name] = {"filename": results}
        
        if prepas_re.search(name) != None:
            results_dict[name]["nivel"] = "preparatoria"
        else:
            results_dict[name]["nivel"] = "universidad"

In [2]:
def get_section_center(html_soup, *args):
    get_section = args[3]
    section = []
    tmp = [i for i in args]
    center = html_soup.find_all("center")
    capture = False
    for c in center:
        # print c.contents
        if c.contents[0].name == "b" and c.contents[0].contents[0] == get_section:
            capture = True
            continue
        elif c.contents[0].name == "b":
            capture = False
        
        if capture and matricula.search(c.contents[0].strip()):
            section.append([int(c.contents[0])] + tmp)
        else:
            capture = False
    
    return section

In [9]:
import os
import re
from bs4 import BeautifulSoup

cupo_re = re.compile("Cupo")
matricula = re.compile("2015[0-9]{5}")
prepas_re = re.compile("PREPARATORIA|PREP|BACHILLERATO|URBANAS")
aceptados_re = re.compile("LISTA DE CANDIDATOS A INSCRIPCION|HIJOS DE TRABAJADORES BUAP")
no_aceptados_re = re.compile("LISTA DE NO ACEPTADOS")

# Segunda opción para obtener lista de datos
index_source = open('index.html')
index_soup = BeautifulSoup(index_source.read())
index_source.close()

# universidades = [{"type": "universidad", "name": uni.contents[0]} for uni in index_soup.select("a[onclick]")]
all = []
for section in index_soup.select("ul")[1:]:
    section_name = section.find("h3").contents[0]
    # print("## " + section_name)
    # Obtiene las secciones, las carreras y los archivos asociados a cada uno
    for p in section.find_all("a"):
        # print(p.contents)
        file_name = ""
        
        if p["href"] != "javascript:void(0)":
            file_name = p["href"]
            file_name = file_name[len("publicacion") + 1:]
        else:
            file_name = p.attrs['onclick'].split("'")[1]
            file_name = file_name[len("publicacion") + 1:]
        
        file_name_tmp = open(file_name)
        content_soup = BeautifulSoup(file_name_tmp.read(), "html.parser")
        file_name_tmp.close()
        
        print(p.contents[0] + "," + file_name + "," + section_name)
            
        # obtener los cupos
        cupos = content_soup.find_all("h1")
        if  cupos != []:
            for i in cupos:
                if cupo_re.search(i.contents[0]):
                    print(i.contents[0].strip().replace("Cupo: ", ""))
        else:
            for cupo in content_soup.find_all("td"):
                if cupo_re.search(cupo.contents[0]) != None:
                    print(cupo.contents[0].strip().replace("Cupo: ", ""))
        
        if len(content_soup.find_all("table")) > 0 and prepas_re.search(p.contents[0]) == None and p.contents[0] != 'TECNICO EN MUSICA':
            status = ["no aceptado", "aceptado"]
            meritos = get_section_center(content_soup, "", "", "", "DISTINCION AL MERITO ACADEMICO",'aceptado', p.contents[0].encode("utf-8"), section_name.encode("utf-8"), file_name)
            all = all + meritos
            # print meritos
            hijos_trabajador_buap = get_section_center(content_soup, "", "", "", "HIJOS DE TRABAJADORES BUAP", 'aceptado', p.contents[0].encode("utf-8"), section_name.encode("utf-8"), file_name)
            all = all + hijos_trabajador_buap
            # print hijos_trabajador_buap
            for table in content_soup.find_all("table"):
                for row in table.find_all("tr"):
                    data = []
                    for cell in row.find_all("td"):
                        try:
                            data.append(int(cell.contents[0]))
                        except:
                            # print(cell.contents[0])
                            continue
                            
                    if len(data) > 0:
                        if status[-1] == "aceptado":
                            data.append("LISTA DE CANDIDATOS A INSCRIPCION")
                        else:
                            data.append("LISTA DE NO ACEPTADOS")
                        data.append(status[-1])
                        data.append(p.contents[0].encode("utf-8"))
                        data.append(section_name.encode("utf-8"))
                        data.append(file_name.encode("utf-8"))
                        all.append(data)
                status.pop()
                
        else:
            print file_name
            print len(content_soup.find_all("table"))
            table = content_soup.find_all("table")[0]
            status = "aceptado"
            lugar = None # para obtener las prepas urbanas
            context = ""
            
            for row in table.find_all("tr"):
                data = []
                for cell in row.find_all("td"):
                    if aceptados_re.search(cell.contents[0]):
                        status = "aceptado"
                        context = cell.contents[0]
                        continue
                    elif no_aceptados_re.search(cell.contents[0]):
                        status = "no aceptado"
                        lugar = None
                        context = cell.contents[0]
                        continue

                    prepa_posible = cell.find("small")
                    if prepa_posible != None:
                        lugar = prepa_posible.contents[0]
                        print lugar
                        continue
                    try:
                        data.append(int(cell.contents[0]))
                    except:
                        #print prepa_posible
                        # print(cell.contents[0])
                        continue
                
                if len(data) == 1:
                    data.append("")
                if len(data) > 0:
                    data.append(context.strip())
                    data.append(status)
                    if lugar != "PREPARATORIAS URBANAS" and lugar != None and file_name == "prepUrbanas.html":
                        data.append(lugar.encode("utf-8"))
                    else:
                        data.append(p.contents[0].encode("utf-8"))

                    data.append(section_name.encode("utf-8"))
                    data.append(file_name.encode("utf-8"))
                    all.append(data)
                    

                   
print(len(all))
all[255]

 BACHILLERATO 5 DE MAYO,BCM.html,PREPARATORIAS
90
BCM.html
1
 URBANAS (CIUDAD DE PUEBLA),prepUrbanas.html,PREPARATORIAS
750
810
350
640
440
280
Cupo:
prepUrbanas.html
1
PREP. 2 DE OCTUBRE DE 1968
PREP. ALFONSO CALDERON MORENO
PREP. EMILIANO ZAPATA
PREP. LIC. BENITO JUAREZ GARCIA
PREP. URB ENRIQUE CABRERA B.
PREP.GRAL. LAZARO CARDENAS
PREPARATORIAS URBANAS
PREPARATORIA REGIONAL ACAJETE,RPA.html,PREPARATORIAS
90
RPA.html
1
PREPARATORIA REGIONAL  CD. SERDAN,RCS.html,PREPARATORIAS
120
RCS.html
1
PREPARATORIA REGIONAL CHIAUTLA DE TAPIA,RPC.html,PREPARATORIAS
60
RPC.html
1
PREPARATORIA REGIONAL CUETZALAN,RCU.html,PREPARATORIAS
140
RCU.html
1
PREPARATORIA REGIONAL  IZUCAR DE MATAMOROS,RIM.html,PREPARATORIAS
105
RIM.html
1
PREPARATORIA REGIONAL  LIBRES,RPL.html,PREPARATORIAS
250
RPL.html
1
PREPARATORIA REGIONAL SIMON BOLIVAR ATLIXCO,RSB.html,PREPARATORIAS
560
RSB.html
1
PREPARATORIA REGIONAL SAN MARTIN TEXMELUCAN,RMT.html,PREPARATORIAS
90
RMT.html
1
PREPARATORIA REGIONAL  TEPEACA,RPT.html,PREP

[201510391,
 773,
 u'LISTA DE CANDIDATOS A INSCRIPCION',
 'aceptado',
 'PREP. 2 DE OCTUBRE DE 1968',
 'PREPARATORIAS',
 'prepUrbanas.html']

In [22]:
print all[23000]
prepas_tecnico = []

for i in all:
    if i[0] == 201525020:
        print len(i)
        
for i in all:
    if len(i) == 7 and i[5] != "PREPARATORIAS":
        prepas_tecnico.append(i)
        
print len(prepas_tecnico)
print prepas_tecnico[0]
print prepas_tecnico[6]
print prepas_tecnico[-35]
    

[201518919, '', '', '', 'DISTINCION AL MERITO ACADEMICO', 'aceptado', u'INGENIERIA EN MECATRONICA', u'AREA DE INGENIERIA Y CIENCIAS EXACTAS', u'LIM-2009.html']
7
335
[201506120, '', u'HIJOS DE TRABAJADORES BUAP', 'aceptado', u'TECNICO EN MUSICA', u'CARRERA T\xc9CNICA', u'TMU.html']
[201500808, 940, u'LISTA DE CANDIDATOS A INSCRIPCION', 'aceptado', u'TECNICO EN MUSICA', u'CARRERA T\xc9CNICA', u'TMU.html']
[201522945, 617, u'LISTA DE NO ACEPTADOS', 'no aceptado', u'TECNICO EN MUSICA', u'CARRERA T\xc9CNICA', u'TMU.html']


In [8]:
from bs4 import BeautifulSoup
urbanas = open("prepUrbanas.html")
urbanas_soup = BeautifulSoup(urbanas.read())
urbanas.close()

table = urbanas_soup.find_all("table")[0]
# print (table.contents)
for row in table.contents:
    if row.name == "tr":
        for cell in row.contents:
            if cell.name == "td":
                for content in cell.contents:
                    if content.name == "h2":
                        print content.contents

[<small>PREP. 2 DE OCTUBRE DE 1968</small>]
[<small>PREP. ALFONSO CALDERON MORENO</small>]
[<small>PREP. EMILIANO ZAPATA</small>]
[<small>PREP. LIC. BENITO JUAREZ GARCIA</small>]
[<small>PREP. URB ENRIQUE CABRERA B.</small>]
[<small>PREP.GRAL. LAZARO CARDENAS</small>]
[<small>PREPARATORIAS URBANAS</small>]


In [107]:
from bs4 import BeautifulSoup
import re

matricula = re.compile("2015[0-9]{5}")

fisio = open('LFT-2009.html')
fisio_soup = BeautifulSoup(fisio.read())
fisio.close()

center = fisio_soup.find_all("center")
capture = False
for c in center:
    # print c.contents
    if c.contents[0].name == "b" and c.contents[0].contents[0] == "DISTINCION AL MERITO ACADEMICO":
        capture = True
        continue
    elif c.contents[0].name == "b":
        capture = False
        
    if capture and matricula.search(c.contents[0].strip()):
        print c.contents
    else:
        capture = False
        
        

[u'201500651']
[u'201504438']
[u'201506148']
[u'201519874']
[u'201521968']
[u'201526920']
[u'201528668']
[u'201536353']
[u'201542005']
[u'201545435']
[u'201546261']
[u'201547810']
[u'201549764']
[u'201551880']
[u'201552434']


In [9]:
for student in all:
    if student[0] in [201500046, 201562829, 201556208, 201502986, 201533135, 201500248, 201559364, 201509402]:
        print student

[201502986, '', u'HIJOS DE TRABAJADORES BUAP', 'aceptado', u'PREP. 2 DE OCTUBRE DE 1968', u'PREPARATORIAS', u'prepUrbanas.html']
[201533135, '', u'HIJOS DE TRABAJADORES BUAP', 'aceptado', u'PREP. 2 DE OCTUBRE DE 1968', u'PREPARATORIAS', u'prepUrbanas.html']
[201500248, 834, u'LISTA DE CANDIDATOS A INSCRIPCION', 'aceptado', u'PREP. 2 DE OCTUBRE DE 1968', u'PREPARATORIAS', u'prepUrbanas.html']
[201559364, 753, u'LISTA DE CANDIDATOS A INSCRIPCION', 'aceptado', u'PREP. 2 DE OCTUBRE DE 1968', u'PREPARATORIAS', u'prepUrbanas.html']
[201509402, '', u'HIJOS DE TRABAJADORES BUAP', 'aceptado', u'PREP. ALFONSO CALDERON MORENO', u'PREPARATORIAS', u'prepUrbanas.html']
[201562829, '', '', '', 'HIJOS DE TRABAJADORES BUAP', 'aceptado', u'DISE\xd1O GRAFICO (TEHUACAN)', u'UNIDAD REGIONAL TEHUACAN', u'RTG-2009.html']
[201500046, 799, 871, 835, 'LISTA DE CANDIDATOS A INSCRIPCION', 'aceptado', u'DISE\xd1O GRAFICO (TEHUACAN)', u'UNIDAD REGIONAL TEHUACAN', u'RTG-2009.html']
[201556208, 607, 495, 551, 'LISTA 

In [15]:
import csv
import os

path = "../datasets/"
if not os.path.exists(path):
    os.makedirs(path)

uni_header = ["matricula", "PAA","PAC", "TOTAL", "perfil", "estado de solicitud", "carrera", "sección", "origen"]
prepa_header = ["matricula", "TOTAL", "perfil", "estado de solicitud", "carrera", "sección", "origen"]
universidad = open( path + "universidades.csv", "w")
carreras_tecnicas = open(path + "carreratecnica.csv", "w")
prepas = open(path + "prepas.csv", "w")

universidad_csv = csv.writer(universidad, delimiter=',')
carreras_tecnicas_csv = csv.writer(carreras_tecnicas, delimiter=',')
prepas_csv = csv.writer(prepas, delimiter=',')

total_prepas = 0
total_unis = 0

count_unis = 0
count_prepas = 0

universidad_csv.writerow(uni_header)
carreras_tecnicas_csv.writerow(prepa_header)
prepas_csv.writerow(prepa_header)

for student in all:
    if len(student) == 7:
        if student[5] != 'PREPARATORIAS':
            carreras_tecnicas_csv.writerow(student)
        else:
            prepas_csv.writerow(student)
            
        total_prepas += 1
        
    if len(student) == 9:        
        universidad_csv.writerow(student)
        total_unis += 1

            
    if student[5] == "aceptado":
        count_unis += 1
    elif student[3] == "aceptado":
        count_prepas += 1
        
print "Universidades aceptados " + str(count_unis) + " de " + str(total_unis)
print "Prepas " + str(count_prepas) + " de " + str(total_prepas)
print "Aproximado de ganacias como base $750 por cada alumno $" + str (len(all) * 750)
print "total de estudiantes " + str(len(all)) + " == " + str(total_unis + total_prepas)

universidad.close()
carreras_tecnicas.close()
prepas.close()

Universidades aceptados 14558 de 32176
Prepas 5865 de 13603
Aproximado de ganacias como base $750 por cada alumno $34334250
total de estudiantes 45779 == 45779
